In [1]:
#import soundfile as sf
import librosa
import numpy as np
import os
import scipy.signal as signal
import musdb

In [2]:
ROOT = '/data/yongzheng/dataset_gen/musdb18/'
NUM_SAMPLES = 5
NUM_SAMPLES_PER_TRAIN = 100
NUM_SAMPLES_PER_TEST = 10
CHUNK = 127 * 768 # 1 second = 44100 fs
SF = 22050

In [3]:
def short_time_fourier_transform(v):
    ft = np.array(librosa.core.stft(v,n_fft=1024,hop_length=768))
    magnitude = np.abs(ft)
    phase = np.angle(ft)
    return magnitude, phase

In [4]:
mus = musdb.DB(root=ROOT, is_wav=False)

In [5]:
subsets = list()

def to_mono(stereo):
    return np.sum(stereo, axis=1)

directories = ['train', 'test']
for subset in directories:
    print("Working on directory {}:".format(subset))
    tracks = mus.load_mus_tracks(subset)
    mix = []
    mix_mag = []
    mix_pha = []
    s1 = []
    s1_mag = []
    s1_pha = []
    s2 = []
    s2_mag = []
    s2_pha = []
    for track in tracks:
        print(" "*4+track.name)
        mono_data = to_mono(track.audio)
        src_1 = to_mono(track.targets['vocals'].audio)
        src_2 = to_mono(track.targets['accompaniment'].audio)
        if subset == 'train':
            NUM_SAMPLES_PER = NUM_SAMPLES_PER_TRAIN
        else:
            NUM_SAMPLES_PER = NUM_SAMPLES_PER_TEST
        print(mono_data.shape, mono_data[:5])
        print(src_1.shape, src_1[:5])
        print(src_2.shape, src_2[:5])
        start_point = np.random.randint(0, high=mono_data.shape[0]-CHUNK, size=NUM_SAMPLES_PER)
        for j in start_point:
            magnitude, phase = short_time_fourier_transform(mono_data[j:j+CHUNK])
            mix.append(mono_data[j:j+CHUNK])
            mix_mag.append(magnitude)
            mix_pha.append(phase)
            magnitude1, phase = short_time_fourier_transform(src_1[j:j+CHUNK])
            s1.append(src_1[j:j+CHUNK])
            s1_mag.append(magnitude1)
            s1_pha.append(phase)
            magnitude2, phase = short_time_fourier_transform(src_2[j:j+CHUNK])
            s2.append(src_2[j:j+CHUNK])
            s2_mag.append(magnitude2)
            s2_pha.append(phase)
    np.save('mix_'+subset,np.array(mix))
    np.save('mix_'+subset+'_mag',np.array(mix_mag))
    np.save('mix_'+subset+'_pha',np.array(mix_pha))
    np.save('s1_'+subset,np.array(s1))
    np.save('s1_'+subset+'_mag',np.array(s1_mag))
    np.save('s1_'+subset+'_pha',np.array(s1_pha))
    np.save('s2_'+subset,np.array(s2))
    np.save('s2_'+subset+'_mag',np.array(s2_mag))
    np.save('s2_'+subset+'_pha',np.array(s2_pha))

In [6]:
"""directories = ['train', 'test']
for i in range(len(directories)):
    print("Working on directory {}:".format(directories[i]))
    curr_dir = os.path.join(ROOT, directories[i])
    mix = []
    mix_mag = []
    mix_pha = []
    s1 = []
    s1_mag = []
    s1_pha = []
    s2 = []
    s2_mag = []
    s2_pha = []
    for f in os.listdir(curr_dir):
        print(" "*4+f)
        if f[-4:] == '.mp4':
            data, sf = librosa.load(os.path.join(curr_dir, f), sr=SF, mono=False)
            print(" "*4+"data loaded! shape = {}".format(data.shape))
            mono_data, _ = librosa.load(os.path.join(curr_dir, f), sr=SF, mono=True)
            src_1 = data[0]
            src_2 = data[1]
            if directories[i] == 'train':
                NUM_SAMPLES_PER = NUM_SAMPLES_PER_TRAIN
            else:
                NUM_SAMPLES_PER = NUM_SAMPLES_PER_TEST
            
            start_point = np.random.randint(0, high=data.shape[1]-CHUNK, size=NUM_SAMPLES_PER)
            for j in start_point:
                magnitude, phase = short_time_fourier_transform(mono_data[j:j+CHUNK])
                mix.append(mono_data[j:j+CHUNK])
                mix_mag.append(magnitude)
                mix_pha.append(phase)
                magnitude1, phase = short_time_fourier_transform(src_1[j:j+CHUNK])
                s1.append(src_1[j:j+CHUNK])
                s1_mag.append(magnitude1)
                s1_pha.append(phase)
                magnitude2, phase = short_time_fourier_transform(src_2[j:j+CHUNK])
                s2.append(src_2[j:j+CHUNK])
                s2_mag.append(magnitude2)
                s2_pha.append(phase)
    np.save('mix_'+directories[i],np.array(mix))
    np.save('mix_'+directories[i]+'_mag',np.array(mix_mag))
    np.save('mix_'+directories[i]+'_pha',np.array(mix_pha))
    np.save('s1_'+directories[i],np.array(s1))
    np.save('s1_'+directories[i]+'_mag',np.array(s1_mag))
    np.save('s1_'+directories[i]+'_pha',np.array(s1_pha))
    np.save('s2_'+directories[i],np.array(s2))
    np.save('s2_'+directories[i]+'_mag',np.array(s2_mag))
    np.save('s2_'+directories[i]+'_pha',np.array(s2_pha))
    
    print(" "*8+directories[i])
    print(" "*8+"Output_Shape: {}".format(mix[0].shape))
    print(" "*8+"Mag_Shape: {}".format(mix_mag[0].shape))
    print(" "*8+"Phase_Shape: {}".format(mix_pha[0].shape))"""